## Databases 

In [1]:
import os
import pandas as pd
import time

In [2]:
hostname = "127.0.0.1"

In [58]:
hospital = pd.read_json("../data/testData/hospital-sort.json").to_dict("records") 
hospitalAllPaths = pd.read_json("../data/testData/hospitall-allPaths.json").to_dict("list")

In [ ]:
college = pd.read_json("../data/testData/college.json").to_dict("records")

### Mongo 

In [36]:
from pymongo import MongoClient

In [37]:
client = MongoClient(host=hostname,port=27017)

In [38]:
client

MongoClient(host=['127.0.0.1:27017'], document_class=dict, tz_aware=False, connect=True)

In [39]:
local = client["local"]

In [59]:
allPaths = [dict(name=k, locations=v) for k,v in hospitalAllPaths.items()]

In [54]:
local["hospital"].insert_many(hospital)

In [60]:
local["hospitalPaths"].insert_many(allPaths)

In [ ]:
collegeMongo = local["college"]

In [ ]:
collegeMongo.insert_many(college)

### Neo4j 

In [8]:
from py2neo import Graph, Node, Relationship

In [34]:
txpool = []
for e in hospital:
    start = Node("Hospital",name=e["start_name"],lat=e["start_lat"],lon=e["start_lon"])
    end = Node("Hospital",name=e["end_name"],lat=e["end_lat"],lon=e["end_lon"])
    
    r = Relationship(start,"NEAREST_PATH",end)
    txpool.append(r)
    
    for i in hospitalAllPaths.get(e["start_name"]): 
        #
        destination = Node("Hospital", name=i["end_name"], lat=e["end_lat"], lon=e["end_lon"]) 
        is_connected = Relationship(start, "IS_CONNECTED", destination, distance=i["distance"])
        txpool.append(is_connected)
    else: 
        continue 

In [65]:
unique = set([e["name"] for e in college])

for idx, collegeName in enumerate(unique):
    # Filter 
    pred = list(filter(lambda e: e["name"] == collegeName, college)) 
    getProgramName = list(map(lambda e: Node("ProgramName",name=e["programName"]), pred))
                              
    university = Node("University", name=pred[0]["name"], web=pred[0]["web"], city=pred[0]["city"])    
    
    [txpool.append(Relationship(e,"HAS_CAPACITY",university)) for e in getProgramName]

In [ ]:
import time
graph = Graph(f"bolt://{hostname}:7687", auth=("neo4j", "12345678"))
tx = graph.begin()
start_time = time.time() 
[print(f"Processing {idx} out of {len(txpool)} - {tx.create(e)}") for idx, e in enumerate(txpool)]
print(f"Before commit: {time.time() - start_time}")
graph.commit(tx)
print(f"After commit: {time.time() - start_time}")

Processing 0 out of 13314 - None
Processing 1 out of 13314 - None
Processing 2 out of 13314 - None
Processing 3 out of 13314 - None
Processing 4 out of 13314 - None
Processing 5 out of 13314 - None
Processing 6 out of 13314 - None
Processing 7 out of 13314 - None
Processing 8 out of 13314 - None
Processing 9 out of 13314 - None
Processing 10 out of 13314 - None
Processing 11 out of 13314 - None
Processing 12 out of 13314 - None
Processing 13 out of 13314 - None
Processing 14 out of 13314 - None
Processing 15 out of 13314 - None
Processing 16 out of 13314 - None
Processing 17 out of 13314 - None
Processing 18 out of 13314 - None
Processing 19 out of 13314 - None
Processing 20 out of 13314 - None
Processing 21 out of 13314 - None
Processing 22 out of 13314 - None
Processing 23 out of 13314 - None
Processing 24 out of 13314 - None
Processing 25 out of 13314 - None
Processing 26 out of 13314 - None
Processing 27 out of 13314 - None
Processing 28 out of 13314 - None
Processing 29 out of 133

Processing 246 out of 13314 - None
Processing 247 out of 13314 - None
Processing 248 out of 13314 - None
Processing 249 out of 13314 - None
Processing 250 out of 13314 - None
Processing 251 out of 13314 - None
Processing 252 out of 13314 - None
Processing 253 out of 13314 - None
Processing 254 out of 13314 - None
Processing 255 out of 13314 - None
Processing 256 out of 13314 - None
Processing 257 out of 13314 - None
Processing 258 out of 13314 - None
Processing 259 out of 13314 - None
Processing 260 out of 13314 - None
Processing 261 out of 13314 - None
Processing 262 out of 13314 - None
Processing 263 out of 13314 - None
Processing 264 out of 13314 - None
Processing 265 out of 13314 - None
Processing 266 out of 13314 - None
Processing 267 out of 13314 - None
Processing 268 out of 13314 - None
Processing 269 out of 13314 - None
Processing 270 out of 13314 - None
Processing 271 out of 13314 - None
Processing 272 out of 13314 - None
Processing 273 out of 13314 - None
Processing 274 out o

Processing 490 out of 13314 - None
Processing 491 out of 13314 - None
Processing 492 out of 13314 - None
Processing 493 out of 13314 - None
Processing 494 out of 13314 - None
Processing 495 out of 13314 - None
Processing 496 out of 13314 - None
Processing 497 out of 13314 - None
Processing 498 out of 13314 - None
Processing 499 out of 13314 - None
Processing 500 out of 13314 - None
Processing 501 out of 13314 - None
Processing 502 out of 13314 - None
Processing 503 out of 13314 - None
Processing 504 out of 13314 - None
Processing 505 out of 13314 - None
Processing 506 out of 13314 - None
Processing 507 out of 13314 - None
Processing 508 out of 13314 - None
Processing 509 out of 13314 - None
Processing 510 out of 13314 - None
Processing 511 out of 13314 - None
Processing 512 out of 13314 - None
Processing 513 out of 13314 - None
Processing 514 out of 13314 - None
Processing 515 out of 13314 - None
Processing 516 out of 13314 - None
Processing 517 out of 13314 - None
Processing 518 out o

Processing 732 out of 13314 - None
Processing 733 out of 13314 - None
Processing 734 out of 13314 - None
Processing 735 out of 13314 - None
Processing 736 out of 13314 - None
Processing 737 out of 13314 - None
Processing 738 out of 13314 - None
Processing 739 out of 13314 - None
Processing 740 out of 13314 - None
Processing 741 out of 13314 - None
Processing 742 out of 13314 - None
Processing 743 out of 13314 - None
Processing 744 out of 13314 - None
Processing 745 out of 13314 - None
Processing 746 out of 13314 - None
Processing 747 out of 13314 - None
Processing 748 out of 13314 - None
Processing 749 out of 13314 - None
Processing 750 out of 13314 - None
Processing 751 out of 13314 - None
Processing 752 out of 13314 - None
Processing 753 out of 13314 - None
Processing 754 out of 13314 - None
Processing 755 out of 13314 - None
Processing 756 out of 13314 - None
Processing 757 out of 13314 - None
Processing 758 out of 13314 - None
Processing 759 out of 13314 - None
Processing 760 out o

Processing 973 out of 13314 - None
Processing 974 out of 13314 - None
Processing 975 out of 13314 - None
Processing 976 out of 13314 - None
Processing 977 out of 13314 - None
Processing 978 out of 13314 - None
Processing 979 out of 13314 - None
Processing 980 out of 13314 - None
Processing 981 out of 13314 - None
Processing 982 out of 13314 - None
Processing 983 out of 13314 - None
Processing 984 out of 13314 - None
Processing 985 out of 13314 - None
Processing 986 out of 13314 - None
Processing 987 out of 13314 - None
Processing 988 out of 13314 - None
Processing 989 out of 13314 - None
Processing 990 out of 13314 - None
Processing 991 out of 13314 - None
Processing 992 out of 13314 - None
Processing 993 out of 13314 - None
Processing 994 out of 13314 - None
Processing 995 out of 13314 - None
Processing 996 out of 13314 - None
Processing 997 out of 13314 - None
Processing 998 out of 13314 - None
Processing 999 out of 13314 - None
Processing 1000 out of 13314 - None
Processing 1001 out

...

Processing 3283 out of 13314 - None
Processing 3284 out of 13314 - None
Processing 3285 out of 13314 - None
Processing 3286 out of 13314 - None
Processing 3287 out of 13314 - None
Processing 3288 out of 13314 - None
Processing 3289 out of 13314 - None
Processing 3290 out of 13314 - None
Processing 3291 out of 13314 - None
Processing 3292 out of 13314 - None
Processing 3293 out of 13314 - None
Processing 3294 out of 13314 - None
Processing 3295 out of 13314 - None
Processing 3296 out of 13314 - None
Processing 3297 out of 13314 - None
Processing 3298 out of 13314 - None
Processing 3299 out of 13314 - None
Processing 3300 out of 13314 - None
Processing 3301 out of 13314 - None
Processing 3302 out of 13314 - None
Processing 3303 out of 13314 - None
Processing 3304 out of 13314 - None
Processing 3305 out of 13314 - None
Processing 3306 out of 13314 - None
Processing 3307 out of 13314 - None
Processing 3308 out of 13314 - None
Processing 3309 out of 13314 - None
Processing 3310 out of 13314

Processing 3513 out of 13314 - None
Processing 3514 out of 13314 - None
Processing 3515 out of 13314 - None
Processing 3516 out of 13314 - None
Processing 3517 out of 13314 - None
Processing 3518 out of 13314 - None
Processing 3519 out of 13314 - None
Processing 3520 out of 13314 - None
Processing 3521 out of 13314 - None
Processing 3522 out of 13314 - None
Processing 3523 out of 13314 - None
Processing 3524 out of 13314 - None
Processing 3525 out of 13314 - None
Processing 3526 out of 13314 - None
Processing 3527 out of 13314 - None
Processing 3528 out of 13314 - None
Processing 3529 out of 13314 - None
Processing 3530 out of 13314 - None
Processing 3531 out of 13314 - None
Processing 3532 out of 13314 - None
Processing 3533 out of 13314 - None
Processing 3534 out of 13314 - None
Processing 3535 out of 13314 - None
Processing 3536 out of 13314 - None
Processing 3537 out of 13314 - None
Processing 3538 out of 13314 - None
Processing 3539 out of 13314 - None
Processing 3540 out of 13314

Processing 3742 out of 13314 - None
Processing 3743 out of 13314 - None
Processing 3744 out of 13314 - None
Processing 3745 out of 13314 - None
Processing 3746 out of 13314 - None
Processing 3747 out of 13314 - None
Processing 3748 out of 13314 - None
Processing 3749 out of 13314 - None
Processing 3750 out of 13314 - None
Processing 3751 out of 13314 - None
Processing 3752 out of 13314 - None
Processing 3753 out of 13314 - None
Processing 3754 out of 13314 - None
Processing 3755 out of 13314 - None
Processing 3756 out of 13314 - None
Processing 3757 out of 13314 - None
Processing 3758 out of 13314 - None
Processing 3759 out of 13314 - None
Processing 3760 out of 13314 - None
Processing 3761 out of 13314 - None
Processing 3762 out of 13314 - None
Processing 3763 out of 13314 - None
Processing 3764 out of 13314 - None
Processing 3765 out of 13314 - None
Processing 3766 out of 13314 - None
Processing 3767 out of 13314 - None
Processing 3768 out of 13314 - None
Processing 3769 out of 13314

Processing 3972 out of 13314 - None
Processing 3973 out of 13314 - None
Processing 3974 out of 13314 - None
Processing 3975 out of 13314 - None
Processing 3976 out of 13314 - None
Processing 3977 out of 13314 - None
Processing 3978 out of 13314 - None
Processing 3979 out of 13314 - None
Processing 3980 out of 13314 - None
Processing 3981 out of 13314 - None
Processing 3982 out of 13314 - None
Processing 3983 out of 13314 - None
Processing 3984 out of 13314 - None
Processing 3985 out of 13314 - None
Processing 3986 out of 13314 - None
Processing 3987 out of 13314 - None
Processing 3988 out of 13314 - None
Processing 3989 out of 13314 - None
Processing 3990 out of 13314 - None
Processing 3991 out of 13314 - None
Processing 3992 out of 13314 - None
Processing 3993 out of 13314 - None
Processing 3994 out of 13314 - None
Processing 3995 out of 13314 - None
Processing 3996 out of 13314 - None
Processing 3997 out of 13314 - None
Processing 3998 out of 13314 - None
Processing 3999 out of 13314

Processing 4202 out of 13314 - None
Processing 4203 out of 13314 - None
Processing 4204 out of 13314 - None
Processing 4205 out of 13314 - None
Processing 4206 out of 13314 - None
Processing 4207 out of 13314 - None
Processing 4208 out of 13314 - None
Processing 4209 out of 13314 - None
Processing 4210 out of 13314 - None
Processing 4211 out of 13314 - None
Processing 4212 out of 13314 - None
Processing 4213 out of 13314 - None
Processing 4214 out of 13314 - None
Processing 4215 out of 13314 - None
Processing 4216 out of 13314 - None
Processing 4217 out of 13314 - None
Processing 4218 out of 13314 - None
Processing 4219 out of 13314 - None
Processing 4220 out of 13314 - None
Processing 4221 out of 13314 - None
Processing 4222 out of 13314 - None
Processing 4223 out of 13314 - None
Processing 4224 out of 13314 - None
Processing 4225 out of 13314 - None
Processing 4226 out of 13314 - None
Processing 4227 out of 13314 - None
Processing 4228 out of 13314 - None
Processing 4229 out of 13314

In [24]:
graph.run("MATCH (n) RETURN COUNT(n)")

COUNT(n)
158


In [12]:
#graph.run("MATCH (n) DETACH DELETE(n)")

(No data)